### Web Crawiling for whiskybase data, with the following objective
#### 1. getting whisky details (object information)
#### 2. getting user details (subject similarity)

In [4]:
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import re

import os
os.chdir(r'C:\Users\thewa\Desktop\Nigel\Whisky')

In [5]:
#Login
username = "nigel_022001@yahoo.com.hk"
password = "Nigel123"

# Set up the webdriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get("https://www.whiskybase.com/account/login?p=%2F")

# Find the username/email field and send the username to the input field.
uname = driver.find_element(by=By.XPATH, value='/html/body/div[1]/div[1]/div/div[3]/form/div[1]/div/input')
uname.send_keys(username)
pword = driver.find_element(by=By.XPATH, value='/html/body/div[1]/div[1]/div/div[3]/form/div[2]/div/input')
pword.send_keys(password)

# Click sign in button to login the website.
button = driver.find_element(by=By.XPATH, value='/html/body/div[1]/div[1]/div/div[3]/form/div[3]/div/button')
button.click()



====== WebDriver manager ======
Current google-chrome version is 113.0.5672
Get LATEST chromedriver version for 113.0.5672 google-chrome
Driver [C:\Users\thewa\.wdm\drivers\chromedriver\win32\113.0.5672.63\chromedriver.exe] found in cache


In [ ]:
'''
#Extract all distiller, whisky url and the whisky country map

distiller_origin = dict()
distiller_url = list()

whisky_distiller_url = 'https://www.whiskybase.com/whiskies/distilleries'
driver.get(whisky_distiller_url)

whisky_distiller_crawl = driver.find_elements(By.XPATH, '//*[@id="compositor-material"]/table/tbody/tr')
for whisky_distiller in whisky_distiller_crawl:
    distiller_vote = 0
    try: 
        distiller_vote = int(whisky_distiller.find_elements(By.TAG_NAME, "td")[3].text)    
        distiller_name = whisky_distiller.find_elements(By.TAG_NAME, "td")[0].text    
        distiller_country = whisky_distiller.find_elements(By.TAG_NAME, "td")[1].text       
        distiller_origin[distiller_name] = distiller_country
        link = whisky_distiller.find_elements(By.TAG_NAME, "a")[0].get_attribute('href')
        distiller_url.append(link)
    except ValueError:
        pass    

whisky_bottle_url = list()

for distiller in distiller_url:
    print(distiller)
    driver.get(distiller)
    
    whisky_bottle_crawl = driver.find_elements(By.XPATH, '//*[@id="compositor-material"]/table/tbody/tr')
    
    for whisky_bottle in whisky_bottle_crawl:    
        try:
            bottle_vote = whisky_bottle.find_elements(By.TAG_NAME, "td")[7]      
            number = float(bottle_vote.text)   
            link = whisky_bottle.find_elements(By.CLASS_NAME, "clickable")[0].get_attribute('href')
            whisky_bottle_url.append(link)
        except:
              pass    

distiller_origin
whisky_bottle_url

#Export to csv to save the link
whisky_url = pd.DataFrame(whisky_bottle_url)
whisky_url.to_csv('whisky_bottle_url.csv')
'''

In [6]:
def crawl_whisky_details(url:str, driver):
    driver.get(url)

    SCROLL_PAUSE_TIME = 1
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        # Scroll down to bottom, Wait to load page, Calculate new scroll height and compare with last scroll height
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(SCROLL_PAUSE_TIME)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height



    # Extract the basic whisky information and score
    bottle_name = driver.find_element(by = By.XPATH, value='/html/body/div[1]/div[1]/div/div[2]/div[1]/div[1]/div[1]/div[1]/div/header/h1').text
    number_of_rating = driver.find_element(by = By.XPATH, value='/html/body/div[1]/div[1]/div/div[2]/div[1]/div[2]/div[1]/div/div[1]/dl/dd[2]').text
    average_rating = driver.find_element(by = By.XPATH, value='//*[@id="partial-aggregate-rating"]/dl/dd[1]/span[1]').text
    average_price = driver.find_element(by = By.XPATH, value='//*[@id="partial-aggregate-rating"]/dl/dd[1]/span[1]').text
    information = [tasting_note.text for tasting_note in driver.find_elements(By.XPATH, "/html/body/div[1]/div[1]/div/div[2]/div[1]/div[1]/div[2]/div[1]/dl/dd")]
    origin = [tasting_note.text for tasting_note in driver.find_elements(By.XPATH, "//ul[@class='breadcrumb']")][0]
    #origin = re.sub(r'(?<![A-Z\W])(?=[A-Z])', ' ', origin[0])

    # Extract user comment and score
    user_names_list = [user_names.text for user_names in driver.find_elements(By.XPATH, "//div[@class='wb--user-list']//th[@scope='row']")]
    tasting_score_list = [tasting_score.text for tasting_score in driver.find_elements(By.XPATH, "//span[@class='wb--note-refinement-rating-value']")] #it is in form of nose, taste, finish
    overall_comment = [tasting_note.text for tasting_note in driver.find_elements(By.XPATH, "//div[@data-translation-field='message']")]
    nose_comment = [nose_note.text for nose_note in driver.find_elements(By.XPATH, "//div[@data-translation-field='nose_text']")]
    taste_comment = [taste_note.text for taste_note in driver.find_elements(By.XPATH, "//div[@data-translation-field='taste_text']")]
    finish_comment = [finish_note.text for finish_note in driver.find_elements(By.XPATH, "//div[@data-translation-field='finish_text']")]

    # Extract TASTINGTAGS
    flavour_list  = [flavour.text for flavour in driver.find_elements(By.XPATH, "//a[@class='btn-tastingtag']")]
    flavour_vote  = [flavour.get_attribute('data-num') for flavour in driver.find_elements(By.XPATH, "//a[@class='btn-tastingtag']")]

    whisky_dict = {'name':bottle_name, 'no_of_rating':number_of_rating, 'rating':average_rating, 
                   'price':average_price, 'full_info': information, 'origin': origin,                    
                   'tasting_score': tasting_score_list,
                   'overall_comment_list': overall_comment,
                   'nose_comment_list': nose_comment, 
                   'taste_comment_list': taste_comment, 
                   'finish_comment_list': finish_comment, 
                   'flavour_dict': dict(zip(flavour_list,flavour_vote)) }

    return whisky_dict, user_names_list

scotish_whisky_region = ['Campbeltown', 'Highlands', 'Islands', 'Islay', 'Lowlands', 'Speyside'] #for changing origin mapping
whisky_bottle_url = pd.read_csv('whisky_bottle_url.csv')
whisky_test_dict, user_list = crawl_whisky_details(whisky_bottle_url.iloc[0,1], driver)

In [7]:
#loop for whisky_url and extract the data
import pickle 
whisky_data = []
full_user_list = []

count = 1
whisky_bottle_url = pd.read_csv('whisky_bottle_url.csv')

for i in range(500):
#for i in range(len(whisky_bottle_url)):        
    whisky_bottle_dict, user_list = crawl_whisky_details(whisky_bottle_url.iloc[i,1], driver)
    whisky_data.append(whisky_bottle_dict)
    full_user_list.append(user_list)
    print(i)

    if count % 100 == 0:
        print("Extracting for number = {}.... ".format(count))
        whisky_address = 'whisky_bottle_data_{}.pkl'.format(count)        
        with open(whisky_address, 'wb') as f:
            pickle.dump(whisky_data, f)
            whisky_data = []
        
        user_address = 'user_name_{}.pkl'.format(count)
        unique_user_list = set([item for sublist in full_user_list for item in sublist])
        with open(user_address, 'wb') as f:
            pickle.dump(unique_user_list, f)
            full_user_list = []
        
        count = count + 1

whisky_address = 'whisky_bottle_data_{}.pkl'.format(count)
user_address = 'user_name_{}.pkl'.format(count)
with open(whisky_address, 'wb') as f:
    pickle.dump(whisky_data, f)
    whisky_data = []

with open(user_address, 'wb') as f:
    pickle.dump(unique_user_list, f)
    full_user_list = []


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54


In [27]:
whisky_data[-1]

({'name': "'t Koelschip 03-year-old",
  'no_of_rating': '',
  'rating': '43.14',
  'price': '43.14',
  'full_info': ['WB37870',
   'Single Malt',
   "'t Koelschip",
   'Distillery Bottling',
   'Janz Whisky',
   '03 years old',
   'Oak Casks',
   '40.0 % Vol.',
   '700 ml',
   '19 nov 2012 7:49 pm',
   ''],
  'origin': "ExploreNetherlands't KoelschipJanz Whisky",
  'tasting_score': [],
  'overall_comment_list': ['Ideal whisky to mix with cola. A lot of cola.',
   '"Single malt gelagerd op 4 soorten houten vaten: Amerikaanse eik, Franse eik, Nederlandse eik en Hongaarse eik. Allemaal nieuwe vaten waardoor de whisky een mooie zachte afdronk heeft."\n\nThis whisky is really not my thing. Have tasted it during the make-your-own-whisky workshop, which was good fun but the intro took to long, the actual making of your own whisky almost felt like a rush job (I went for 50% toasted oak, 25% French oak, and 25% American oak and will see how it turns out in 3 years time). All things said, had go

In [12]:
'''
Function to get user info (rewrite)
'''

def get_user_preference(userid, driver):
    'result is a list of whisky details list'
    #open user page
    user_url = 'https://www.whiskybase.com/profile/' + userid + '/lists'    
    driver.get(user_url)

    #extract bottle details
    user_wine_list = driver.find_elements(By.XPATH, "/html/body/div[1]/div[1]/div/div[4]/table/tbody/tr")
    user_wine_details = []  #user, whisky, rating, my rating

    for wine in user_wine_list :        
        wine_details_cols = wine.find_elements(By.TAG_NAME, "td")        
        dummy_list = [userid, wine_details_cols[1].text, wine_details_cols[8].text, wine_details_cols[9].text]
        user_wine_details.append(dummy_list)        
    return user_wine_details

#test case
user = '72HEAVEN'
get_user_preference(user,driver)


[['72HEAVEN', "Aberlour A'bunadh batch #45", '86.77', '-'],
 ['72HEAVEN', 'Bowmore 1990 Wy Earth, Wind, Fire!', '90.43', '-'],
 ['72HEAVEN', 'Bunnahabhain 1989 KCWS Kings Court Choice', '88.07', '-'],
 ['72HEAVEN', 'Caol Ila 1990 ST Single Cask', '87.60', '-'],
 ['72HEAVEN', 'Caol Ila 2008 3W Best shared with friends', '87.71', '-'],
 ['72HEAVEN', 'Eye Land Pork 1998 TWJ', '87.20', '-'],
 ['72HEAVEN', 'Girvan 1991 3W Best shared with friends', '88.00', '-'],
 ['72HEAVEN', 'Glen Grant 1998 Ch7 Monologue', '88.00', '-'],
 ['72HEAVEN',
  'Glen Keith 1991 DD The Nectar of the Daily Drams',
  '87.21',
  '-'],
 ['72HEAVEN', 'Glendronach 1995 Single Cask', '89.89', '-'],
 ['72HEAVEN', "Glenlossie 1992 BR Berrys'", '85.73', '-'],
 ['72HEAVEN', 'Highland Park 1998 MoS', '90.04', '-'],
 ['72HEAVEN',
  "Ichiro's MWR Ichiro's Malt - Mizunara Wood Reserve",
  '83.19',
  '-'],
 ['72HEAVEN', 'Ireland 1989 DD Anniversary Edition - Volume 6', '89.39', '-'],
 ['72HEAVEN', 'Isle of Jura 1988 TWhm', '87.8

In [ ]:
'''
Attempt to modulize info
'''

class whisky:
    def __init__(self, name, age):
        self.name = name
        self.age = age
    
    def myfunc(self):
        print("Hello my name is " + self.name)

p1 = Person("John", 36)

print(p1.name)
print(p1.age)

{"'t Koelschip": 'Netherlands',
 '1000 Hills Distillery': 'Rwanda',
 '10th Mountain': 'United States',
 '10th Street Distillery': 'United States',
 '1769 Distillery Inc.': 'Canada',
 '23rd Street Distillery': 'Australia',
 '2bar Spirits': 'United States',
 '3 Badge Beverage': 'United States',
 '3 Howls': 'United States',
 '45th Parallel': 'United States',
 '66 Gilead Distillery': 'Canada',
 '8 Doors Distillery': 'Scotland',
 'A. Roborel de Climens': 'France',
 'A. Smith Bowman': 'United States',
 'Aber Falls Whisky Distillery': 'United Kingdom',
 'Aberfeldy': 'Scotland',
 'Aberlour': 'Scotland',
 'Abhainn Dearg': 'Scotland',
 'Acha': 'Spain',
 'Ad Gefrin': 'United Kingdom',
 'Adams Distillery': 'Australia',
 'Adelaide Hills': 'Australia',
 'Adnams (Copper House Distillery)': 'United Kingdom',
 'Aecht Schlenkerla': 'Germany',
 'Agárdi Pálinkafőzde': 'Hungary',
 'Agder Brenneri': 'Norway',
 'AGITATOR': 'Sweden',
 'Aichinger': 'Austria',
 'Ailsa Bay': 'Scotland',
 'Aioi Unibio Co. Ltd.': 

In [1]:
len(whisky_url)

NameError: name 'whisky_url' is not defined